In [1]:
import tensorflow as tf
from tensorflow.layers import dense, flatten, conv2d, max_pooling2d
from utils import reshape_train_var
import gym
import numpy as np
from rewards import discount_rewards
from memory import MemoryBuffer, MTMemoryBuffer
from policies import VPGTrainer, VAPGTrainer
from threading import Thread
import time
import math
from multiprocessing import Process
from env import *

In [2]:
class PPOTrainer():
    def __init__(self, in_op, out_op, value_out_op, act_type='discrete', sess=None, clip_val=0.2, ppo_iters=30,
                 v_iters=30, actor_lr=3e-4, value_lr=1e-3):
        """
        Create a wrapper for RL networks for easy training.
        Args:
            in_op (tf.Placeholder): Observation input to architecture
            out_op (tf.Variable): Action output of architecture
            act_type (string): 'discrete' for a discrete actions space or 'continuous'
                               for a continuous actions space
            sess (tf.Session): A session if you would like to use a custom session,
                               if left none it will be automatically created
        """

        if not sess:
            self.renew_sess()
        
        self.in_op = in_op
        self.out_op = out_op
        self.value_out_op = value_out_op
        self._prev_weights = None
        self.clip_val = clip_val
        self.ppo_iters = ppo_iters
        self.actor_lr=3e-4 
        self.value_lr=1e-3
        
        if act_type in ('discrete', 'd'):
            self.train = self._create_discrete_trainer()
            self.act_type = 'discrete'
        elif act_type in ('continuous', 'c'):
            self.train = self._create_continuous_trainer()
            self.act_type = 'continuous'
        else:
            raise TypeError('act_type must be \'discrete\' or \'continuous\'')
        
    def renew_sess(self):
        """
        Starts a new internal Tensorflow session
        """
        self.sess = tf.Session()
        
    def end_sess(self):
        """
        Ends the internal Tensorflow session if it exists
        """
        if self.sess:
            self.sess.close()
        
    def _create_discrete_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a discrete action space
        """
        # First passthrough
        
        self.act_holders = tf.placeholder(tf.int64, shape=[None])
        self.reward_holders = tf.placeholder(tf.float64, shape=[None])
        
        self.act_masks = tf.one_hot(self.act_holders, self.out_op.shape[1].value, dtype=tf.float64)
        self.resp_acts = tf.reduce_sum(self.act_masks *  self.out_op, axis=1)
        print(self.resp_acts)
        
        self.advantages = self.value_out_op - self.reward_holders
        print(self.advantages)
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float64, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float64, shape=self.resp_acts.shape)
 
        self.policy_ratio = tf.reduce_mean(self.resp_acts / self.old_prob_holders)
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        
        self.actor_optimizer = tf.train.AdamOptimizer(learning_rate=self.actor_lr)
        self.value_optimizer = tf.train.AdamOptimizer(learning_rate=self.value_lr)
        
        # Initial update
        
        self.initial_loss = tf.reduce_mean(-tf.log(self.resp_acts) * self.advantages)
        print(self.initial_loss)
        self.initial_update = self.actor_optimizer.minimize(self.initial_loss)

        # Value update
        
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - self.value_out_op))
        self.value_update = self.value_optimizer.minimize(self.value_loss)
        
        # Actor update
        
        self.actor_loss = tf.reduce_mean(self.min_loss)
        self.actor_update = self.actor_optimizer.minimize(self.actor_loss - self.value_loss)

        
        def update_func(train_data):
            
            self.sess.run([self.initial_update], 
                           feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                      self.act_holders: train_data[:, 1],
                                      self.reward_holders: train_data[:, 2]})
            
            self.sess.run([self.value_update], 
                           feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                      self.act_holders: train_data[:, 1],
                                      self.reward_holders: train_data[:, 2]})
            
            
#             self.old_probs, self.old_advantages = self.sess.run([self.resp_acts, self.advantages], 
#                                     feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                                self.act_holders: train_data[:, 1],
#                                                self.reward_holders: train_data[:, 2]})
            
#             for i in range(self.ppo_iters):
#                 self.sess.run([self.actor_update], 
#                                feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                     self.act_holders: train_data[:, 1],
#                                     self.old_prob_holders: self.old_probs,
#                                     self.advatange_holders: self.old_advantages,
#                                     self.reward_holders: train_data[:, 2]})
            
#                 self.sess.run([self.value_update], 
#                                feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                           self.reward_holders: train_data[:, 2]})

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _create_continuous_trainer(self):
        """
        Creates a function for vanilla policy training with a continuous action space
        """
        # First passthrough
        
        self.reward_holders = tf.placeholder(tf.float64, shape=[None])
        
        self.advantages = self.value_out_op - self.reward_holders
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float64, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float64, shape=self.out_op.shape)

        self.policy_ratio = tf.reduce_mean(self.out_op / self.old_prob_holders, axis=1)
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        self.actor_loss = tf.reduce_mean(self.min_loss)

        self.actor_optimizer = tf.train.AdamOptimizer(learning_rate=self.actor_lr)
        self.actor_update = self.actor_optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_optimizer = tf.train.AdamOptimizer(learning_rate=self.value_lr)
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - self.value_out_op))
        self.value_update = self.value_optimizer.minimize(self.value_loss)
        
        def update_func(train_data):
            self.old_probs, self.old_advantages = self.sess.run([self.out_op, self.advantages], 
                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                               self.reward_holders: train_data[:, 2]})
            
            for i in range(self.ppo_iters):
                self.sess.run([self.actor_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.old_prob_holders: self.old_probs,
                                    self.advatange_holders: self.old_advantages})
            
            for i in range(self.ppo_iters):
                self.sess.run([self.value_update], 
                           feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                      self.reward_holders: train_data[:, 2]})

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _gen_discrete_act(self, obs):
        act_probs = self.sess.run(self.out_op, feed_dict={self.in_op: obs})
        act = np.random.choice(list(range(len(act_probs)+1)), p=act_probs[0])
        
        return act
    
    def _gen_continuous_act(self, obs):
        act_vect = self.sess.run(self.out_op, feed_dict={self.in_op: obs})[0]
        
        # TODO: Add gaussian noise t128o action vector
        act_vect = [a + np.random.normal(0., 0.1) for a in act_vect]
        
        return np.array(act_vect)
        
    def gen_act(self, obs):
        if self.act_type == 'discrete':
            return self._gen_discrete_act(obs)
        else:
            return self._gen_continuous_act(obs)
        
    def train(self, obs, rewards, acts):
        raise RuntimeError('The train method was not properly created')

In [3]:
# Cart pole network

env = make_cart_pole()

obs = tf.placeholder(tf.float64, shape=[None, 4])
dense1 = dense(obs, 32, activation=tf.tanh)
dense2 = dense(dense1, 32, activation=tf.tanh)
act_probs = dense(dense2, 2)
softmax_probs = tf.nn.softmax(act_probs)

v_dense1 = dense(obs, 32, activation=tf.tanh)
v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
value = dense(v_dense2, 1)

network = VAPGTrainer(obs, softmax_probs, value, act_type='discrete')

# network = PPOTrainer(obs, softmax_probs, value, act_type='discrete', ppo_iters=80, v_iters=80)

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
# Car race network

# obs = tf.placeholder(tf.float64, shape=np.concatenate([[None], env.observation_space.shape]))
# conv1 = conv2d(obs, 16, [5, 5], padding='valid', activation=tf.tanh)
# maxpool1 = max_pooling2d(conv1, 4, 4)
# conv2 = conv2d(maxpool1, 32, [3, 3], padding='valid', activation=tf.tanh)
# maxpool2 = max_pooling2d(conv2, 2, 2)
# flatten1 = flatten(maxpool2)
# act_vects = dense(flatten1, env.action_space.shape[0])

# network = VanillaPolicyTrainer(obs, act_vects, act_type='continuous')

In [5]:
# Lunar lander discrete network

# obs = tf.placeholder(tf.float64, shape=[None, env.observation_space.shape[0]])
# dense1 = dense(obs, 64, activation=tf.nn.tanh)
# dense2 = dense(dense1, 64, activation=tf.nn.tanh)
# act_probs = dense(dense2, 2)
# softmax_probs = tf.nn.softmax(act_probs)

# network = VanillaPolicyTrainer(obs, softmax_probs, act_type='d')

In [6]:
# # Lunar lander continuous network

# env = make_lunar_lander_c()

# obs = tf.placeholder(tf.float64, shape=[None, env.observation_space.shape[0]])
# dense1 = dense(obs, 32, activation=tf.nn.tanh)
# dense2 = dense(dense1, 32, activation=tf.nn.tanh)
# act_vects = dense(dense2, 2, activation=tf.nn.tanh)

# v_dense1 = dense(obs, 32, activation=tf.nn.tanh)
# v_dense2 = dense(v_dense1, 32, activation=tf.nn.tanh)
# value = dense(v_dense2, 1)

# network = PPOTrainer(obs, act_vects, value, act_type='c', ppo_iters=20)

In [7]:
n_epochs = 1000000
max_steps = 200
update_freq = 4000 # In steps
print_freq = 1

mb = MemoryBuffer(update_freq)

In [8]:
## import time
epoch_steps = 0
total_epochs = 0
epoch_rewards = []

for epoch in range(n_epochs):
    
    epoch_steps = 0
    while epoch_steps < update_freq:
        obs = env.reset()

        episode_reward = 0
        mb.start_rollout()
        for step in range(max_steps):
            act = network.gen_act([obs])
#             env.render()
#             time.sleep(0.02)

            obs_next, rew, d, _ = env.step(act)
            episode_reward += rew

            mb.record(obs, act, rew)
            obs = obs_next

            epoch_steps += 1
            if d:
                break

        epoch_rewards.append(episode_reward)
        total_epochs += 1

    train_data = mb.to_data()
    network.train(train_data)

    if epoch % print_freq == 0 and epoch != 0:
        print(f'Update #{int(epoch * print_freq)}, Recent Reward:', np.mean(epoch_rewards))
        epoch_rewards = []

Update #1, Recent Reward: 25.967637540453076
Update #2, Recent Reward: 28.055944055944057
Update #3, Recent Reward: 23.680473372781066
Update #4, Recent Reward: 26.706666666666667
Update #5, Recent Reward: 26.15032679738562
Update #6, Recent Reward: 21.361702127659573
Update #7, Recent Reward: 19.681372549019606
Update #8, Recent Reward: 21.17989417989418
Update #9, Recent Reward: 18.28054298642534
Update #10, Recent Reward: 18.204545454545453
Update #11, Recent Reward: 16.953389830508474
Update #12, Recent Reward: 17.094017094017094
Update #13, Recent Reward: 17.188841201716738
Update #14, Recent Reward: 15.98804780876494
Update #15, Recent Reward: 15.717647058823529
Update #16, Recent Reward: 15.212121212121213
Update #17, Recent Reward: 14.098245614035088
Update #18, Recent Reward: 14.507246376811594
Update #19, Recent Reward: 13.899653979238755
Update #20, Recent Reward: 13.662116040955631
Update #21, Recent Reward: 13.245033112582782
Update #22, Recent Reward: 13.071895424836601
U

KeyboardInterrupt: 